In [378]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Для задачи Question Answering мы используем класс BertForQuestionAnswering из библиотеки transformers

In [379]:
import requests
import json
import torch
import os
from tqdm import tqdm
import pandas as pd

from transformers import BertTokenizerFast
from torch.utils.data import DataLoader
from transformers import BertForQuestionAnswering
from transformers import AdamW

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [380]:
if not os.path.exists('/content/drive/MyDrive/BERT-SQuAD'):
  os.mkdir('/content/drive/MyDrive/BERT-SQuAD')

# Загрузка данных

In [381]:
# !wget -nc https://github.com/aspushkarev/nlp-test-task-2023/tree/main/data/raw/train.json

# Data preprocessing

In [382]:
# Load the training dataset and take a look at it
with open('train.json', 'rb') as f:
  squad = json.load(f)

In [383]:
squad[0].keys()

dict_keys(['id', 'text', 'label', 'extracted_part'])

In [384]:
squad[18]

{'id': 807692640,
 'text': 'Приложение № 3 к извещению об осуществлении закупки (Электронный аукцион) ПРОЕКТ ГОСУДАРСТВЕНННОГО КОНТРАКТА ГОСУДАРСТВЕННЫЙ КОНТРАКТ N ____ на поставку радиотелефонов г. Иркутск "__" __________ ____ г. Федеральное казенное учреждение «Главное чика об одностороннем отказе от исполнения Контракта вступает в силу и Контракт считается расторгнутым через десять дней с даты надлежащего уведомления заказчиком Поставщика об одностороннем отказе от исполнения Контракта; 4.4.5. При исполнении Контракта по согласованию Заказчика с Поставщиком допускается поставка Товара, качество, технические и функциональные характеристики (потребительские свойства) которого являются улучшенными по сравнению с качеством и соответствующими техническими и функциональными характеристиками, указанными в Контракте; 4.4.6. требовать уплаты неустоек (штрафов, пеней) в соответствии с условиями Контракта; 4.4.7. Стороны обязуются получать почтовые отправления, направляемые друг другу, не позд

In [385]:
def read_data(path):  
  # load the json file
  # with open(path, 'rb') as f:
  #   squad = json.load(f)

  squad = pd.read_json(path)

  contexts = []
  questions = []
  answers = []
  num_q = 0
  num_pos = 0
  num_imp = 0

  for text in squad['text']:
      num_q += 1
      contexts.append(text.lower())

  for question in squad['label']:
      questions.append(question.lower())

  for answer in squad['extracted_part']:
      if answer.get('answer_start') == [0]:
          num_imp += 1
      else:
          num_pos += 1
          answers.append(answer)

  return num_q, num_pos, num_imp, contexts, questions, answers

In [386]:
num_q, num_pos, num_imp, contexts, questions, answers = read_data('train.json')

In [387]:
# print a random question and answer
print(f'There are {len(questions)} questions')
print(questions[-1])
print(answers[-1])

There are 1799 questions
обеспечение гарантийных обязательств
{'text': ['Заказчиком установлено требование обеспечения исполнения договора и (или) обеспечения исполнения гарантийных обязательств. 5% от начальной (максимальной) цены договора, что составляет 27450'], 'answer_start': [1213], 'answer_end': [1402]}


# Токенизация

Загрузим токенайзер

In [388]:
tokenizer = BertTokenizerFast.from_pretrained('oceanpty/mbert-squad')

train_encodings = tokenizer(contexts, questions, truncation=True, padding=True)
# valid_encodings = tokenizer(valid_contexts, valid_questions, truncation=True, padding=True)

In [389]:
type(train_encodings)

transformers.tokenization_utils_base.BatchEncoding

In [390]:
# type(model)

In [391]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [392]:
no_of_encodings = len(train_encodings['input_ids'])
print(f'We have {no_of_encodings} context-question pairs')

We have 1799 context-question pairs


Посмотрим что получили после токенизации данных

In [393]:
print(train_encodings.keys())
print(len(train_encodings['input_ids']))
print(len(train_encodings['input_ids'][0]))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
1799
512


In [394]:
print(train_encodings['input_ids'][0])

[101, 10387, 14149, 26692, 555, 12709, 88174, 10191, 60427, 12470, 66524, 543, 104121, 11075, 46730, 10520, 10234, 86613, 10648, 1779, 10929, 68430, 68073, 77802, 68430, 78675, 102011, 44026, 11211, 13248, 26325, 55407, 44479, 10387, 103149, 104038, 68073, 77802, 68430, 78675, 102011, 44026, 11211, 108318, 62013, 10234, 86613, 10648, 80765, 26198, 65501, 10433, 556, 76983, 558, 97205, 101216, 80765, 10541, 73276, 113, 105940, 50329, 117, 37410, 17010, 13862, 13826, 114, 60427, 13865, 64895, 543, 542, 12118, 74545, 103072, 85308, 10385, 35085, 49867, 10970, 48712, 543, 17164, 65424, 10234, 51642, 10648, 10122, 16635, 543, 10234, 86613, 11557, 117, 541, 11448, 54678, 16616, 16346, 59936, 70135, 10513, 19820, 21137, 10234, 51642, 10648, 10122, 16635, 543, 10234, 86613, 11557, 16087, 23807, 33580, 18197, 61409, 41538, 10364, 10234, 86613, 10648, 543, 27878, 35085, 49867, 10970, 48712, 10880, 10375, 95128, 10384, 16616, 16346, 59936, 117, 23807, 19954, 104722, 27332, 11075, 15888, 119, 1082

In [395]:
# num_of_encodings = len(train_encodings['input_ids'])
# print(f'We have {num_of_encodings} context-question pairs')

Декодируем первую пару вопрос-ответ и взглянем что получается

In [396]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 общая информация номер извещения 0328300032822000806 наименование объекта закупки поставка продуктов питания способ определения поставщика ( подрядчика, исполнителя ) открытый конкурс в бль порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 федерального закона № 44 - фз. выбор способа обеспечения осуществляется участником закупки самостоятельно. срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 федерального закона № 44 - фз. условия независимой гарантии в соответствии со ст. 45 федерального закона № 44 - фз. реквизиты счета в соответств

Далее мы конвертируем символы начала и конца позиций в начало и конец позиции токенов. Так как слова конвертируются в токены, ответ начала и конца показывает индекс начала и конца токена, который содержит ответ, а не конкретные символы в контексте

In [397]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  for i in range(len(answers)):
    start_positions.append(encodings.char_to_token(i, answers[i]['answer_start'][0]))
    end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'][0] - 1))

    # if start position is None, the answer passage has been truncated
    if start_positions[-1] is None:
    # if start_positions[-1] is [0]:
      start_positions[-1] = tokenizer.model_max_length
    if end_positions[-1] is None:
    # if end_positions[-1] is [0]:
      end_positions[-1] = tokenizer.model_max_length

  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, answers)

In [398]:
train_encodings['start_positions'][:10]

[348, 382, 369, 385, 423, 372, 382, 379, 371, 379]

# Определение набора данных (датасета)

Мы определяем наш датасет используя класс PyTorch Dataset из torch.utils и создаём dataloaders

In [399]:
class SQuAD_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [400]:
train_dataset = SQuAD_Dataset(train_encodings)
# valid_dataset = SQuAD_Dataset(valid_encodings)

In [401]:
# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=16)

# Fine-Tuning

Use of AdamW which is a stochastic optimization method that modifies the typical implementation of weight decay in Adam, by decoupling weight decay from the gradient update. This helps to avoid overfitting which is necessary in this case were the model is very complex.

Set the lr=2e-5 as I read that this is the best value for the learning rate for this task.

Загрузим предобученную модель BERT, иcточник https://huggingface.co/oceanpty/mbert-squad

In [402]:
model = BertForQuestionAnswering.from_pretrained("oceanpty/mbert-squad")

In [403]:
# Check on the available device - use GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [404]:
N_EPOCHS = 5
optim = AdamW(model.parameters(), lr=5e-5)

model.to(device)
model.train()

for epoch in range(N_EPOCHS):
  loop = tqdm(train_loader, leave=True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch+1}')
    loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/113 [00:00<?, ?it/s]


IndexError: ignored